In [1]:
# Install dependencies
!pip install flask pyngrok torch transformers vllm

import os
import threading
import time
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
from kaggle_secrets import UserSecretsClient
import torch
from transformers import AutoTokenizer, AutoModel
from vllm import LLM, SamplingParams
import gc
import re
import psutil

# Retrieve secrets
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
ngrok_token = user_secrets.get_secret("NGROK_TOKEN")

# Set Ngrok authtoken
conf.get_default().auth_token = ngrok_token

# Set environment variables to reduce memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128,expandable_segments:True"
os.environ["HF_TOKEN"] = hf_token

# Set CUDA_VISIBLE_DEVICES to map cuda:0 to GPU 1 and cuda:1 to GPU 0
os.environ["CUDA_VISIBLE_DEVICES"] = "1,0"

# Clear GPU memory
def clear_gpu_memory():
    torch.cuda.empty_cache()
    gc.collect()
    print("Cleared GPU memory.")

clear_gpu_memory()

# Load embedding model on cuda:1 (physical GPU 0)
embedding_model_name = "BAAI/bge-en-icl"
embedding_device = torch.device("cuda:1" if torch.cuda.device_count() > 1 else "cuda:0")
try:
    embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name, token=hf_token)
    embedding_model = AutoModel.from_pretrained(
        embedding_model_name,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        token=hf_token
    ).eval().to(embedding_device)
except Exception as e:
    print(f"Failed to load embedding model: {e}")
    raise

# Load generation model with VLLM on cuda:0 (physical GPU 1)
plan_model_name = "meta-llama/Llama-3.2-3B-Instruct"
try:
    plan_llm = LLM(
        model=plan_model_name,
        tensor_parallel_size=1,
        dtype="float16",
        gpu_memory_utilization=0.85,
        max_model_len=4096
    )
    plan_tokenizer = AutoTokenizer.from_pretrained(plan_model_name, token=hf_token)
    if plan_tokenizer.pad_token is None:
        plan_tokenizer.pad_token = plan_tokenizer.eos_token
except Exception as e:
    print(f"Failed to load generation model: {e}")
    raise

# Flask app
app = Flask(__name__)

@app.route('/embed', methods=['POST'])
def embed():
    try:
        data = request.json
        texts = data.get('texts', [])
        max_length = data.get('max_length', 256)
        clear_gpu_memory()
        inputs = embedding_tokenizer(
            ["query: " + text for text in texts],
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=max_length
        )
        inputs = {k: v.to(embedding_device) for k, v in inputs.items()}
        with torch.no_grad(), torch.amp.autocast('cuda'):
            outputs = embedding_model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0]
            norm_embed = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        embeddings_list = norm_embed.cpu().tolist()
        return jsonify({'embeddings': embeddings_list})
    except Exception as e:
        print(f"Embedding error: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/generate', methods=['POST'])
def generate():
    try:
        data = request.json
        prompt = data.get('prompt', '')
        sampling_params = data.get('sampling_params', {})
        # Apply chat template
        formatted_prompt = plan_tokenizer.apply_chat_template(
            [{"role": "system", "content": "You are a doctor speaking to a parent."},
             {"role": "user", "content": prompt}],
            tokenize=False
        )
        # Generate text
        outputs = plan_llm.generate([formatted_prompt], SamplingParams(**sampling_params))
        generated_text = outputs[0].outputs[0].text.strip()
        print(f"Raw vLLM output: {repr(generated_text)}")
        # Robustly strip assistant prefix, role tags, and any vLLM artifacts
        generated_text = re.sub(r'^(?:assistant|user|system|<\|[^>]*\|>)+[\s\n]*', '', generated_text, flags=re.MULTILINE | re.IGNORECASE)
        generated_text = re.sub(r'\n\s*\n+', '\n', generated_text).strip()
        print(f"Processed generated text: {repr(generated_text)}")
        return jsonify({'generated_text': generated_text})
    except Exception as e:
        print(f"Generation error: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/status', methods=['GET'])
def status():
    try:
        memory = psutil.virtual_memory()
        gpu_memory = torch.cuda.memory_allocated(embedding_device) / 1024**2 if torch.cuda.is_available() else 0
        return jsonify({
            'status': 'running',
            'memory_percent': memory.percent,
            'gpu_memory_mb': gpu_memory
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Start Flask app in a separate thread
def run_flask():
    app.run(host='0.0.0.0', port=5000)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Start Ngrok tunnel
public_url = ngrok.connect(5000)
print(f"Ngrok Tunnel URL: {public_url}")

# Keep the notebook running and log status
while True:
    try:
        memory = psutil.virtual_memory()
        print(f"Server status: Memory usage {memory.percent}%")
        time.sleep(60)
    except Exception as e:
        print(f"Status check error: {e}")
        time.sleep(60)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.1/294.1 MB 5.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 53.2 kB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 114.1 kB/s eta 0:00:00a 0:00

2025-04-27 08:30:57.730626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745742657.971577      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745742658.040087      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Cleared GPU memory.


tokenizer_config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Raw vLLM output: "<|start_header_id|>assistant<|end_header_id|>\n\nHello! Due to the lack of specific dosage information for Lorazepam at 50 years old, I recommend starting with a low dose of 0.5-1.0 mg orally, 3 times a day. For Diazepam, a starting dose of 2.5-5.0 mg orally, 2-3 times a day, may be considered. Follow doctor’s advice!\n\nCaution! No interactions have been found between Lorazepam and Diazepam with food or alcohol. However, it's essential to inform your doctor about any medications or substances you're taking, as they may still interact with these medications. Consult a doctor!"
Processed generated text: "Hello! Due to the lack of specific dosage information for Lorazepam at 50 years old, I recommend starting with a low dose of 0.5-1.0 mg orally, 3 times a day. For Diazepam, a starting dose of 2.5-5.0 mg orally, 2-3 times a day, may be considered. Follow doctor’s advice!\nCaution! No interactions have been found between Lorazepam and Diazepam with food or alcohol. Howev

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Raw vLLM output: "<|start_header_id|>assistant<|end_header_id|>\n\nHello! For the patient, I recommend starting with a low dose of Alprazolam, 0.25mg orally, 2-3 times a day, and Clonazepam, 1mg orally, 2-3 times a day. Follow doctor’s advice!\n\nCaution! No significant food or alcohol interactions were found for this patient. However, it's essential to inform the patient to avoid consuming grapefruit products and grapefruit juice, as they may interact with these medications. Consult a doctor!"
Processed generated text: "Hello! For the patient, I recommend starting with a low dose of Alprazolam, 0.25mg orally, 2-3 times a day, and Clonazepam, 1mg orally, 2-3 times a day. Follow doctor’s advice!\nCaution! No significant food or alcohol interactions were found for this patient. However, it's essential to inform the patient to avoid consuming grapefruit products and grapefruit juice, as they may interact with these medications. Consult a doctor!"
Server status: Memory usage 34.3%
Server s

KeyboardInterrupt: 